# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他數值型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#check data dtype
df.dtypes.value_counts()#.apply('count')

object     5
int64      3
float64    2
dtype: int64

In [3]:
df.nunique()

Pclass         3
Name        1307
Sex            2
Age           98
SibSp          7
Parch          8
Ticket       929
Fare         281
Cabin        186
Embarked       3
dtype: int64

In [4]:
#只取類別值 (object) 型欄位, 存於 object_features 中
'''
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
'''
#object_feature
object_features = df.columns[df.dtypes=='object']
print(f'{len(object_features)} Object Features : {object_features}\n')
#numeric_feature
numeric_features = df.columns[df.dtypes!='object']
print(f'{len(numeric_features)} Numeric Features : {numeric_features}\n')

# Missing value handling
df[object_features] = df[object_features].fillna('None')
df[numeric_features] = df[numeric_features].fillna(-1)
train_num = train_Y.shape[0]
df.head()

5 Object Features : Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

5 Numeric Features : Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')



,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [5]:
#Label Encoding + Logistic Regression
LE = LabelEncoder()
df_LE = df.copy()
for col in object_features:
    df_LE[col] = LE.fit_transform(df[col])
#df_LE_num =  pd.concat([df_numeric, df_LE], axis=1)
train_X_LE = df_LE[:train_num]
LR_estimator = LogisticRegression()
start = time.time()
LE_result = cross_val_score(LR_estimator, train_X_LE, train_Y, scoring='accuracy', cv=5)
time_duration = time.time() - start
print(f'score:{LE_result.mean()}\ntime_duration:{time_duration}')
df_LE.head()

score:0.7901426597115313
time_duration:0.09994053840637207


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,155,1,22.0,1,0,720,7.2500,185,3
1,1,286,0,38.0,1,0,816,71.2833,106,0
2,3,523,0,26.0,0,0,914,7.9250,185,3
3,1,422,0,35.0,1,0,65,53.1000,70,3
4,3,22,1,35.0,0,0,649,8.0500,185,3


# 作業2
* 承上題，三者比較效果何者最好?

In [6]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7879396209713713


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,155,1,33,1,0,720,21,185,3
1,0,286,0,58,1,0,816,238,106,0
2,2,523,0,40,0,0,914,51,185,3
3,0,422,0,54,1,0,65,218,70,3
4,2,22,1,54,0,0,649,53,185,3


In [7]:
# 加上 'Cabin' 欄位的計數編碼
df_Cabin_cnt = df.copy()
df_cnt = df_Cabin_cnt.groupby(['Cabin'])['Name'].agg({'Cabin_cnt':'count'}).reset_index()
df_Cabin_cnt = pd.merge(df_Cabin_cnt, df_cnt, on='Cabin', how='left')

df_Cabin_cnt.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_cnt
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,1014
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,1014
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S,1014


In [8]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
LE = LabelEncoder()
for col in object_features:
    df_Cabin_cnt[col] = LE.fit_transform(df_Cabin_cnt[col])
df_Cabin_cnt = df_Cabin_cnt.drop('Cabin', axis=1)
print(df_Cabin_cnt.head())
train_X_Cabin_cnt = df_Cabin_cnt[:train_num]
estimator_Cabin_cnt = LogisticRegression()
start = time.time()
result_Cabin_cnt = cross_val_score(estimator_Cabin_cnt, train_X_Cabin_cnt, train_Y, scoring='accuracy', cv=5)
time_duration = time.time() - start
print(f'score:{result_Cabin_cnt.mean()}\ntime_duration:{time_duration}')

   Pclass  Name  Sex   Age  SibSp  Parch  Ticket     Fare  Embarked  Cabin_cnt
0       3   155    1  22.0      1      0     720   7.2500         3       1014
1       1   286    0  38.0      1      0     816  71.2833         0          2
2       3   523    0  26.0      0      0     914   7.9250         3       1014
3       1   422    0  35.0      1      0      65  53.1000         3          2
4       3    22    1  35.0      0      0     649   8.0500         3       1014
score:0.800242677904395
time_duration:0.07296609878540039


In [10]:
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [9]:
df[['Cabin']].nunique()

Cabin    187
dtype: int64

In [11]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_Cabin_hash  = df.copy()
df_Cabin_hash['Cabin_hash'] = df_Cabin_hash['Cabin'].map(lambda x: hash(x) % 10)

for col in object_features:
    df_Cabin_hash[col] = LE.fit_transform(df_Cabin_hash[col])
df_Cabin_hash = df_Cabin_hash.drop('Cabin', axis=1)
print(df_Cabin_hash.head())
train_X_Cabin_hash = df_Cabin_hash[:train_num]
estimator_Cabin_hash = LogisticRegression()
start = time.time()
result_Cabin_hash = cross_val_score(estimator_Cabin_hash, train_X_Cabin_hash, train_Y, scoring='accuracy', cv=5)
time_duration = time.time() - start
print(f'score:{result_Cabin_hash.mean()}\ntime_dutation:{time_duration}')


   Pclass  Name  Sex   Age  SibSp  Parch  Ticket     Fare  Embarked  \
0       3   155    1  22.0      1      0     720   7.2500         3   
1       1   286    0  38.0      1      0     816  71.2833         0   
2       3   523    0  26.0      0      0     914   7.9250         3   
3       1   422    0  35.0      1      0      65  53.1000         3   
4       3    22    1  35.0      0      0     649   8.0500         3   

   Cabin_hash  
0           3  
1           8  
2           3  
3           6  
4           3  
score:0.7935072400858647
time_dutation:0.07296538352966309


In [12]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_Cabin_cnt_hash = df.copy()
df_tmp = df_Cabin_cnt_hash.groupby('Cabin')['Name'].agg({'Cabin_cnt':'count'}).reset_index()
df_Cabin_cnt_hash = pd.merge(df_Cabin_cnt_hash, df_tmp, on='Cabin', how='left')
for col in object_features:
    df_Cabin_cnt_hash[col] = LE.fit_transform(df_Cabin_cnt_hash[col])
df_Cabin_cnt_hash['Cabin_hash'] = df_Cabin_cnt_hash['Cabin'].map(lambda x: hash(x) % 10)
df_Cabin_cnt_hash = df_Cabin_cnt_hash.drop('Cabin', axis=1)
print(df_Cabin_cnt_hash.head())
train_X_Cabin_cnt_hash = df_Cabin_cnt_hash[:train_num]
estimator_Cabin_cnt_hash = LogisticRegression()
start = time.time()
result_Cabin_cnt_hash = cross_val_score(estimator_Cabin_cnt_hash, train_X_Cabin_cnt_hash,
                                        train_Y, scoring='accuracy', cv=5)
time_duration = time.time() - start
print(f'score:{result_Cabin_cnt_hash.mean()}\ntime_duration:{time_duration}')

   Pclass  Name  Sex   Age  SibSp  Parch  Ticket     Fare  Embarked  \
0       3   155    1  22.0      1      0     720   7.2500         3   
1       1   286    0  38.0      1      0     816  71.2833         0   
2       3   523    0  26.0      0      0     914   7.9250         3   
3       1   422    0  35.0      1      0      65  53.1000         3   
4       3    22    1  35.0      0      0     649   8.0500         3   

   Cabin_cnt  Cabin_hash  
0       1014           5  
1          2           6  
2       1014           5  
3          2           0  
4       1014           5  
score:0.7980016221083366
time_duration:0.06396341323852539
